# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../output_data/cities.csv')
weather_df

,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed
0,Fortuna,US,40.60,-124.16,1607636329,55.99,73,14,1.99
1,Bluff,NZ,-46.60,168.33,1607636329,50.00,77,93,11.99
2,Dunedin,NZ,-45.87,170.50,1607636287,44.01,87,100,28.99
3,Pochutla,MX,15.74,-96.47,1607636329,84.20,51,5,11.41
4,Majene,ID,-3.54,118.97,1607636329,78.53,83,100,5.03
...,...,...,...,...,...,...,...,...,...
548,Wasilla,US,61.58,-149.44,1607636424,21.99,73,90,1.28
549,Geraldton,AU,-28.77,114.60,1607636424,62.60,93,75,3.36
550,Saravena,CO,6.95,-71.88,1607636425,79.07,80,62,0.31
551,Airai,TL,-8.93,125.41,1607636425,64.26,95,95,1.30


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
weather_df = weather_df.fillna(0)
humidity = weather_df["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = weather_df.loc[(weather_df['Max Temp'] >= 85) & (weather_df['Wind Speed'] <= 10) & (weather_df['Cloudiness'] <= 20)].dropna()
ideal_weather

,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed
208,Bangui,CF,4.36,18.55,1607636364,86.00,70,20,2.57
397,Paraná,AR,-31.73,-60.52,1607636399,93.00,30,0,8.01
430,Castaños,MX,26.78,-101.42,1607636405,88.00,10,11,1.99
466,Luján,AR,-34.57,-59.10,1607636411,98.01,42,0,3.00
505,Presidente Franco,PY,-25.53,-54.62,1607636417,93.20,38,20,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather.loc[:, ['City' , 'Country' , 'Lat' , 'Lng']]
hotel_df["Hotel"] = ''
hotel_df

,City,Country,Lat,Lng,Hotel
208,Bangui,CF,4.36,18.55,
397,Paraná,AR,-31.73,-60.52,
430,Castaños,MX,26.78,-101.42,
466,Luján,AR,-34.57,-59.10,
505,Presidente Franco,PY,-25.53,-54.62,


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "hotel",  
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    params['location'] = f'{lat},{lng}'
    
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    try:
        print(f"Closest hotel in {city} is {response[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel'] = response[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    
hotel_df

Retrieving Results for Index 208: Bangui.
Missing field/result... skipping.
Retrieving Results for Index 397: Paraná.
Missing field/result... skipping.
Retrieving Results for Index 430: Castaños.
Missing field/result... skipping.
Retrieving Results for Index 466: Luján.
Missing field/result... skipping.
Retrieving Results for Index 505: Presidente Franco.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel
208,Bangui,CF,4.36,18.55,
397,Paraná,AR,-31.73,-60.52,
430,Castaños,MX,26.78,-101.42,
466,Luján,AR,-34.57,-59.10,
505,Presidente Franco,PY,-25.53,-54.62,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marked_loc = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marked_loc)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))